In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from compmec import nurbs
from helper import getH, getD, solve_system

# Introdução

No notebook anterior tratamos EDPs de diferentes tipos, todos lineares. Mas a mema abordagem utilizada não funciona para o problema que trataremos aqui: resolver a Equação de Navier Stokes.

Antes disso, introduzimos ao problema. Teremos as quantidades:

* Velocidade do fluido: $\mathbf{u} = \left(u_1, \ u_2, \ u_3\right) = \left(u, \ v, \ w\right)$
* Pressão do fluido: $p$
* Fator de campo: $\mathbf{g}$
* Densidade: $\rho$
* Fluido newtoniano de viscosidade: $\mu$
* Tempo: $t$
* Espaço: $\left(x, \ y, \ z\right) = \left(x_1, \ x_2, \ x_3\right)$

Resolver a equação de Navier Stokes se resume a encontrar os valores da velocidade $\mathbf{u}$ e pressão $p$ ao longo do domínio $\Omega_{xyzt}$.

Na notação vetorial temos a equação de balanço de massa dado por $(1.1)$ e o balanço de momento dado por $(1.2)$

$$\dfrac{\partial \rho}{\partial t} + \nabla \cdot \left(\rho \mathbf{u}\right) = 0 \tag{1.1}$$

$$\dfrac{\partial \left(\rho \mathbf{u}\right)}{\partial t} + \nabla \cdot \left( \rho \mathbf{u} \otimes \mathbf{u} \right) = -\nabla p + \rho \mathbf{g} + \mu \nabla^2 \mathbf{u}\tag{1.2}$$

Na notação indicial, $(1.1)$ e $(1.2)$ se tornam $(1.3)$ e $(1.4)$

$$\dfrac{\partial \rho}{\partial t} + \sum_{i = 1}^{3} \dfrac{\partial \left(\rho u_i\right)}{\partial x_i} = 0
\tag{1.3}$$
$$\dfrac{\partial \rho u_i}{\partial t} + \sum_{j=1}^{3} \dfrac{\partial \left(\rho u_i u_j\right)}{\partial x_j} = -\dfrac{\partial p}{\partial x_i} + \rho g_i + \mu \sum_{j=1}^{3} \dfrac{\partial^2 u_i}{\partial x_j^2} \ \ \ \ \ \forall i \tag{1.4}$$

Para o primeiro exemplo, simplificaremos e utilizaremos:

* Malha bidimensional: $\vec{u} = (u, \ v, \ 0)$
* Fluido incompressível: $\rho = \text{const}$
* Força de campo nulas: $\mathbf{g} = \mathbf{0}$.

Também fazemos a adimensionalização do problema, resultando em $(1.5)$ e $(1.6)$:

$$\begin{align*}\nabla \cdot \vec{u} & = 0 \tag{1.5} \\ \dfrac{\partial \vec{u}}{\partial t} + \left(\vec{u} \cdot \nabla\right) \vec{u} & = -\nabla p + \mu \nabla^2 \vec{u} \tag{1.6} 
\end{align*}$$

Aqui chamaremos de $\mu$ como o inverso do número de Reynolds, apenas para facilitar a notação. Não confundir com o $\mu$ da viscosidade da Equação $(1.2)$.




Na malha $\Omega_{xyt} = \left[0, \ 1\right] \times \left[0, \ 1\right] \times \left[0, \ \infty\right)$.

# Diferentes formulações

Embora tenhamos já apresentado as equações, há diferentes de tipos de formulação com diferentes variáveis. Antes de mostra-las, vamos enunciar funções auxiliares:

* Linhas de corrente $\psi$ 

   Assumindo que as velocidades $u$ e $v$ satisfazem

   $$u = \dfrac{\partial \psi}{\partial y} \ \ \ \ \ \ \ \ \ \ \ \ \ v =  -\dfrac{ \partial  \psi}{ \partial x} \tag{2.1}$$
   $$\dfrac{\partial u}{\partial x} + \dfrac{\partial v}{\partial y} = \dfrac{\partial}{\partial x}\left(\dfrac{\partial \psi}{\partial y}\right) + \dfrac{\partial}{\partial y}\left(-\dfrac{\partial \psi}{\partial x}\right) = 0$$

   Então adotar $\psi$ já satifaz a Equação $(1.5)$

* Vorticidade $w$:

   Como a Equação $(1.6)$ envolve o gradiente da pressão, e que é um dos grandes problemas em fluidos, uma solução é elimina-lo tomando o rotacional da Equação $(1.6)$.

   $$w = \nabla \times \vec{u} = \dfrac{\partial v}{\partial x} - \dfrac{\partial u}{\partial y} \tag{2.2}$$
   $$\dfrac{\partial w}{\partial t} + \left(u \cdot \nabla \right) w = \mu \nabla^2 w \tag{2.3}$$

1. Formulação de variáveis $(u, \ v, \ p)$

   A primeira delas é utilizar as variáveis padrões, que expandindo as Equações $(1.5)$ e $(1.6)$ obteremos: 

   $$\begin{align*}\dfrac{\partial u}{\partial x} + \dfrac{\partial v}{\partial y} & = 0 \\ \dfrac{\partial u}{\partial t} + u \dfrac{\partial u}{\partial x} + v\dfrac{\partial u}{\partial y} & = -\dfrac{\partial p}{\partial x} + \mu \left(\dfrac{\partial^2 u}{\partial x^2}+\dfrac{\partial^2 u}{\partial y^2}\right)\\ \dfrac{\partial v}{\partial t} + u\dfrac{\partial v}{\partial x} + v\dfrac{\partial v}{\partial y} & = -\dfrac{\partial p}{\partial y} + \mu \left(\dfrac{\partial^2 v}{\partial x^2}+\dfrac{\partial^2 v}{\partial y^2}\right)  \end{align*}$$

   Nessa formulação temos $3$ equações e $3$ variáveis a serem descobertas.

2. Formulação de variáveis $(\psi, \ p)$

   Satisfaz-se automaticamente a Equação $(1.5)$ de forma que reformulamos o sistema para:

   $$\begin{align*}\dfrac{\partial^2 \psi }{\partial t \partial y} + \dfrac{\partial \psi}{\partial y} \dfrac{\partial^2 \psi}{\partial x \partial y} -\dfrac{\partial \psi}{\partial x} \dfrac{\partial^2 \psi}{\partial y^2} & = -\dfrac{\partial p}{\partial x} + \mu \left(\dfrac{\partial^3 \psi}{\partial x^2 \partial y}+\dfrac{\partial^3 \psi}{\partial y^3}\right)  \\ -\dfrac{\partial^2 \psi}{\partial t \partial x} - \dfrac{\partial \psi}{\partial y}\dfrac{\partial^2 \psi}{\partial x^2} + \dfrac{\partial \psi}{\partial x}\dfrac{\partial^2 \psi }{\partial x \partial y} & = -\dfrac{\partial p}{\partial y} - \mu \left(\dfrac{\partial^3 \psi}{\partial x^3}+\dfrac{\partial^3 \psi}{\partial x\partial y^2}\right)\end{align*}$$

   Nessa formulação temos $2$ equações e $2$ variáveis a serem descobertas.

3. Formulação de variáveis $(u, \ v, \ w)$

   Fazendo o rotacional ($\nabla \times$) da Equação $(1.6)$, juntando com a Equação $(1.5)$ e $(2.2)$ teremos o sistema

   $$\begin{align*}\dfrac{\partial u}{\partial x} + \dfrac{\partial v}{\partial y} & = 0 \\ w & = \dfrac{\partial v}{\partial x} - \dfrac{\partial u}{\partial y}\\ \dfrac{\partial w}{\partial t} + u \dfrac{\partial w}{\partial x} + v\dfrac{\partial w}{\partial y} & = \mu \left(\dfrac{\partial^2 w}{\partial x^2}+\dfrac{\partial^2 w}{\partial y^2}\right)\\  \end{align*}$$

   Nessa formulação temos $3$ equações e $3$ variáveis a serem descobertas.

3. Formulação de variáveis $(u, \ v)$

   Fazendo o rotacional ($\nabla \times$) da Equação $(1.6)$, juntando com a Equação $(1.5)$ e $(2.2)$ teremos o sistema

   $$\begin{align*}\dfrac{\partial u}{\partial x} + \dfrac{\partial v}{\partial y} & = 0 \\  \dfrac{\partial }{\partial t}\left(\dfrac{\partial v}{\partial x} - \dfrac{\partial u}{\partial y} \right) + \text{non-linear} & = \mu \cdot \text{diffusion}  \end{align*}$$

   Com

   $$\begin{align*}\text{diffusion} & = \dfrac{\partial^3 v}{\partial x^3}-\dfrac{\partial^3 u}{\partial x^2 \partial y}+\dfrac{\partial^3 v}{\partial x\partial y^2}-\dfrac{\partial^3 u}{\partial y^3} \\ \text{non-linear} & = u \dfrac{\partial^2 v}{\partial x^2} - u \dfrac{\partial^2 u}{\partial x \partial y} + v\dfrac{\partial^2 v}{\partial x\partial y} - v\dfrac{\partial^2 u}{\partial y^2}  \\ \end{align*}$$

   Nessa formulação temos $2$ equações e $2$ variáveis a serem descobertas.

5. Formulação de variáveis $(\psi, \ w)$

   Do mesmo modo que a formulação 3, mas substituimos $u$ e $v$ pelos da Equação $(2.1)$, resultando em

   $$\begin{align*}w + \dfrac{\partial^2 \psi}{\partial x^2} + \dfrac{\partial^2 \psi}{\partial y^2} & = 0 \\ \dfrac{\partial w}{\partial t} + \dfrac{\partial \psi}{\partial y} \dfrac{\partial w}{\partial x} - \dfrac{\partial \psi}{\partial x} \dfrac{\partial w}{\partial y} & = \mu \left(\dfrac{\partial^2 w}{\partial x^2}+\dfrac{\partial^2 w}{\partial y^2}\right)\\  \end{align*}$$

   Nessa formulação temos $2$ equações e $2$ variáveis a serem descobertas.

6. Formulação na variável $(\psi)$

   Pegando a formulação 5, basta substituir a primeira equação na segunda para obter

   $$\dfrac{\partial}{\partial t}\left(\dfrac{\partial^2 \psi}{\partial x^2}+ \dfrac{\partial^2 \psi}{\partial y^2}\right) = \text{non-linear} + \mu \cdot \text{diffusion}$$

   Com 

   $$\begin{align*}\text{diffusion} & = \dfrac{\partial^4 \psi}{\partial x^4} + 2\dfrac{\partial^4 \psi}{\partial x^2 \partial y^2} + \dfrac{\partial^4 \psi}{\partial y^4} \\ \text{non-linear} & = \dfrac{\partial \psi}{\partial x} \dfrac{\partial}{\partial y}\left(\dfrac{\partial^2 \psi}{\partial x^2}+\dfrac{\partial^2 \psi}{\partial y^2}\right) - \dfrac{\partial \psi}{\partial y}\dfrac{\partial}{\partial x}\left(\dfrac{\partial^2 \psi}{\partial x^2}+\dfrac{\partial^2 \psi}{\partial y^2}\right) \\ & =  \dfrac{\partial \psi}{\partial x} \dfrac{\partial^3 \psi}{\partial y^3} - \dfrac{\partial \psi}{\partial y}\dfrac{\partial^3 \psi}{\partial x \partial y^2} + \dfrac{\partial \psi}{\partial x} \dfrac{\partial^3 \psi}{\partial x^2\partial y} -\dfrac{\partial \psi}{\partial y}\dfrac{\partial^3 \psi}{\partial x^3}\end{align*}$$

   
   Nessa formulação temos apenas $1$ equação e $1$ variáveis a ser descoberta.

# Condições de contorno

As condições de contorno determinam o valor correto do problema. Através dela que se é informado ao interior as informações das paredes.

* Condição de contorno das velocidades
    Para o problema da cavidade, que trataremos aqui, temos as condições de não-penetração do fluido:

    $$\vec{v} \cdot \vec{n} = \vec{0} \ \ \ \ \ \text{on} \ \partial\Omega$$

    Dessa equação se tira que

    $$u(0, \ y) = u(1, \ y) = v(x, \ 0) = v(x, \ 1) = 0$$

    Além disso, tem a condição de velocidade tangencial. Como as paredes lateriais e abaixo não se movem, então teremos

    $$v(0, \ y) = v(1, \ y) = u(x, \ 0) = 0$$

    A última informação vem da parede de cima, que diz que

    $$u(x, \ 1) = U_{top}(x)$$

    A condição de contorno depende das variáveis utilizadas. As formulamos apenas para $(u, \ v)$, mas é de interesse saber a condição de contorno de outras variáveis (em caso de usar outras formulações).

* Condição de contorno das linhas de corrente
    Como mostrado [aqui](https://math.stackexchange.com/questions/4633857/how-to-transform-the-boundary-conditions-between-formulations-of-pde-2d-fluid-f), teremos a condição de contorno para:

    * Em toda borda, apenas uma linha de corrente, devido à condição de não-penetração
        $$\psi(x, \ y) = 0 \ \ \ \ \ \text{on} \ \partial  \Omega$$
    * Na parede esquerda, direita e em baixo, a velocidade tangencial é nula
        $$\left[\dfrac{\partial \psi}{\partial x}\right]_{x=0} = \left[\dfrac{\partial \psi}{\partial x}\right]_{x=1} = \left[\dfrac{\partial \psi}{\partial y}\right]_{y=0} = 0$$
    * Na parede de cima, a velocidade tangencial é dada 
        $$\left[\dfrac{\partial \psi}{\partial y}\right]_{y=1} = U_{top}(x)$$

* Condição de contorno da pressão

    Como mostrado [aqui](https://math.stackexchange.com/questions/4572257/pressure-values-at-the-boundary-in-navier-stokes-equations), a formulação para pressão é dada por

    $$\nabla^2 p = - \nabla \cdot \left(\vec{u} \cdot \nabla\right) \vec{u} \ \ \ \ \ \ \text{on} \ \partial \Omega$$

    ou uma maneira mais simples de se dizer, é simplesmente adotar

    $$\dfrac{\partial p}{\partial n} = 0 \ \ \ \ \ \ \text{on} \ \partial \Omega$$

* Condição de contorno da vorticidade

    Embora para os outros métodos a condição de contorno é algo numérico, de valor, para vorticidade ela depende de outra vorticidade

    * Utilizando a velocidade,
        Como $w = \nabla \times \vec{v}$ então

        Na parede esquerda e direita
        $$w(0, \ y) = \left[\dfrac{\partial v}{\partial x}\right]_{x=0} \ \ \ \ \ \ \ \ \ \ \ \ \ \ w(1, \ y)  = \left[\dfrac{\partial v}{\partial x}\right]_{x=1} $$

        Em cima e em baixo
        $$w(x, \ 0) = - \left[\dfrac{\partial u}{\partial y}\right]_{y=0} \ \ \ \ \ \ \ \ \ \ \ w(x, \ 1)  = - \left[\dfrac{\partial u}{\partial y}\right]_{y=1}$$


    * Utilizando linhas de corrente

        Como $w + \nabla^2 \psi = 0$, então

        $$w(0, y) = -\dfrac{\partial^2 \psi}{\partial x^2} \ \ \ \ \ \ \ \ \ w(1, y) = -\dfrac{\partial^2 \psi}{\partial x^2}$$
        $$w(x, 0) = -\dfrac{\partial^2 \psi}{\partial y^2} \ \ \ \ \ \ \ \ \ w(x, 1) = -\dfrac{\partial^2 \psi}{\partial y^2}$$

# Estratégia com elementos finitos

Embora tenhamos diversas formulações, é recomendado utilizar elementos finitos apenas no espaço.

Isto é, se temos uma função de variáveis espaciais $\mathbf{x}$ e temporais $t$, por exemplo $u(\mathbf{x}, \ t)$, então supomos 

$$u(\mathbf{x}, \ t) = \sum_{i} \varphi_{i}(\mathbf{x}) \cdot u_{i}(t)$$

Em que $\varphi_i(\mathbf{x})$ são as funções de base no espaço, que serão utilizadas para integrar no espaço.

A partir dessa formulação, teremos uma equação EDO com uma derivada temporal, como por exemplo na Equação de Poisson

$$\dfrac{\partial u}{\partial t} + \nabla^2 u = 0$$
$$\varphi_{j} \cdot \dot{u}_{j} + \left(\nabla^2 \varphi_{j}\right) \cdot u_{j} = 0$$
$$\left(\int_{\Omega} \phi \varphi \ d\Omega\right) \dot{\mathbf{U}} + \left(\int_{\Omega} \phi \nabla^2 \varphi \ d\Omega\right) \mathbf{U} = \mathbf{0} $$
$$\mathbf{A} \cdot \dot{\textbf{U}} + \mathbf{B} \cdot \mathbf{U} = \mathbf{0}$$

Pode-se utilizar então um integrador temporal para resolver tal equação. O mais simples de todos é de Euler explicito

$$\mathbf{A} \cdot \left(\dfrac{\textbf{U}_{k+1}-\textbf{U}_k}{\Delta t}\right) + \mathbf{B} \cdot \mathbf{U}_{k} = \mathbf{0}$$

# Formulação usando linhas de corrente (6)

Como primeiro exemplo, vamos utilizar a EDP com a variável de linhas de corrente $\psi$. Por conveniência, chamaremos de $s:= \psi$. Dessa forma

$$\left(\dfrac{\partial^2 }{\partial x^2}+ \dfrac{\partial^2 }{\partial y^2}\right) \dfrac{\partial s}{\partial t} = \text{non-linear} + \mu \cdot \left(\dfrac{\partial^4 }{\partial x^4} + 2\dfrac{\partial^4 }{\partial x^2 \partial y^2} + \dfrac{\partial^4 }{\partial y^4}\right)s$$
$$\text{non-linear} =  \dfrac{\partial s}{\partial x} \dfrac{\partial^3 s}{\partial y^3} - \dfrac{\partial s}{\partial y}\dfrac{\partial^3 s}{\partial x \partial y^2} + \dfrac{\partial s}{\partial x} \dfrac{\partial^3 s}{\partial x^2\partial y} -\dfrac{\partial s}{\partial y}\dfrac{\partial^3 s}{\partial x^3}$$

Escrevemos

$$s(x, \ y, \ t) = \sum_{a, \ b} N_{a,px}(x) \cdot N_{b, py}(y) \cdot S_{a,b}(t)$$

Fazendo a formulação de elementos finitos teremos:

$$\int_{\Omega} N_{i}N_{j} \left(\dfrac{\partial^2 }{\partial x^2}+ \dfrac{\partial^2 }{\partial y^2}\right) \dfrac{\partial s}{\partial t} \ d\Omega = \underbrace{\left(\mathbf{H}_{x02}\mathbf{H}_{y00} + \mathbf{H}_{x00}\mathbf{H}_{y02}\right)}_{\mathbf{A}} \cdot \dot{\mathbf{S}}$$
$$\int_{\Omega} N_{i}N_{j} \left(\dfrac{\partial^4 }{\partial x^4}+ 2\dfrac{\partial^4 }{\partial x^2\partial y^2}+ \dfrac{\partial^4 }{\partial y^4}\right) s \ d\Omega = \underbrace{\left(\mathbf{H}_{x04}\mathbf{H}_{y00} +\mathbf{H}_{x02}\mathbf{H}_{y02}+ \mathbf{H}_{x00}\mathbf{H}_{y04}\right)}_{\mathbf{D}} \cdot \mathbf{S}$$
$$\int_{\Omega} N_{i} N_{j} \cdot \text{non-linear} \ d\Omega = \underbrace{\left(\mathbf{H}_{x010}\mathbf{H}_{y003} +\mathbf{H}_{x001}\mathbf{H}_{y012}+ \mathbf{H}_{x012}\mathbf{H}_{y001}+\mathbf{H}_{x003}\mathbf{H}_{y010}\right)}_{\mathbf{B}} \cdot \mathbf{S}\cdot \mathbf{S}$$

$$\mathbf{A} \cdot \dot{\mathbf{S}} = \mathbf{B} \cdot \mathbf{S} \cdot \mathbf{S} + \mathbf{D} \cdot \mathbf{S}$$



### Utilizando Runge-Kutta

O algoritmo de Runge-Kutta pode ser utilizado. Como temos

$$\dot{\mathbf{S}} = \underbrace{\mathbf{A}^{-1} \left(\mathbf{B} \cdot \mathbf{S} \cdot \mathbf{S} + \mathbf{D} \cdot \mathbf{S}\right)}_{\mathbf{F(\mathbf{S})}}$$

Então iteramos

$$\begin{align*}d\mathbf{S}_1 & = \Delta t \cdot \mathbf{F}\left(\mathbf{S}_{k}\right) \\ d\mathbf{S}_2 & = \Delta t \cdot \mathbf{F}\left(\mathbf{S}_{k} + \dfrac{1}{2} d\mathbf{S}_1\right) \\ d\mathbf{S}_3 & = \Delta t \cdot \mathbf{F}\left(\mathbf{S}_{k}+\dfrac{1}{2}d\mathbf{S}_2\right) \\ d\mathbf{S}_4 & = \Delta t \cdot \mathbf{F}\left(\mathbf{S}_{k}d\mathbf{S}_3\right) \\ \mathbf{S}_{k+1} & = \mathbf{S}_{k} + \dfrac{1}{6}\left(d\mathbf{S}_1+2d\mathbf{S}_2+2d\mathbf{S}_3+d\mathbf{S}_4\right) \end{align*}$$




### Utilizando um método implicito
Partindo de 

$$\mathbf{A} \cdot \dot{\mathbf{S}} = \mathbf{B} \cdot \mathbf{S} \cdot \mathbf{S} + \mathbf{D} \cdot \mathbf{S}$$

Fazemos

$$\mathbf{A} \cdot \left(\dfrac{\mathbf{S}_{k+1}-\mathbf{S}_{k-1}}{2\Delta t}\right) = \mathbf{B} \cdot \mathbf{S}_{k} \cdot \mathbf{S}_{k} + \mathbf{D} \cdot \mathbf{S}_{k+1}$$